# Preliminaries

### Import libraries

In [ ]:
 import numpy as np
 # Fix randomness and hide warnings
seed = 0

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import ResNet50 as ResNet50
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.layers import Flatten, Dense
from tensorflow.keras.applications.resnet50 import preprocess_input

print(tf.__version__)

2.14.0


In [ ]:
# Import other libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns

### Load Dataset

In [ ]:
dataset=np.load('clean_data.npz',allow_pickle=True)
data = dataset['data']
labels = dataset['labels']

[[[ 20.  38.   0.]

  [ 44.  62.  22.]

  [151. 173. 127.]

  ...

  [177. 220. 149.]

  [180. 224. 149.]

  [181. 225. 148.]]



 [[ 18.  35.   1.]

  [ 12.  29.   0.]

  [ 85. 106.  65.]

  ...

  [177. 220. 149.]

  [179. 223. 148.]

  [179. 223. 146.]]



 [[ 12.  23.   6.]

  [  9.  22.   2.]

  [  1.  17.   0.]

  ...

  [171. 213. 147.]

  [170. 213. 142.]

  [170. 213. 141.]]



 ...



 [[  3.  13.   2.]

  [  6.  16.   5.]

  [ 11.  18.  10.]

  ...

  [101. 139.  82.]

  [ 83. 115.  66.]

  [ 81. 113.  64.]]



 [[  6.  17.   0.]

  [  8.  19.   2.]

  [ 15.  23.   8.]

  ...

  [ 87. 125.  68.]

  [ 98. 130.  80.]

  [ 90. 122.  73.]]



 [[  6.  18.   0.]

  [  8.  20.   0.]

  [ 19.  27.  12.]

  ...

  [ 79. 117.  60.]

  [106. 138.  88.]

  [101. 133.  84.]]]

healthy


In [ ]:
#count occurences for each class
setsize_before = labels.shape[0]
ClassCount = pd.DataFrame(labels, columns=['state'])['state'].value_counts()
healthyCount = ClassCount.at['healthy']
unhealthyCount = ClassCount.at['unhealthy']

print('Counting occurrences of target classes BEFORE DATA AUGMENTATION:')
print("healthy samples occurence =", healthyCount)
print("unhealthy samples occurence =", unhealthyCount)

print('\nTotal number of samples : ', setsize_before)

Counting occurrences of target classes BEFORE DATA AUGMENTATION:

healthy samples occurence = 3101

unhealthy samples occurence = 1903



Total number of samples :  5004


# Data augmentation

### Create augmented images from the underrepresented class (unhealthy)

In [ ]:
def img_augmentation(MyImgSet, MyLabels) :

    augmentation = tfk.Sequential([
        tfkl.RandomRotation(0.1, seed = 42),
        tfkl.RandomFlip(mode="horizontal", seed=42 ),
        tfkl.RandomFlip(mode="vertical", seed = 42)
    ], name='preprocessing')

    MyImgSetLength = MyImgSet.shape[0]

    L = [augmentation(MyImgSet[i,:,:,:]) for i in range(MyImgSetLength) if MyLabels[i]=='unhealthy']

    return(np.array(L))

# create a vector with an augmented version of every single unhealthy image
unhealthy_augmented = img_augmentation(data, labels)

print('number of augmented samples created ', unhealthy_augmented.shape[0])

#set the number of augmented data to create to balance both classes in the dataset
N = healthyCount - unhealthyCount
print('number of selected images : ', N)

#create an array with random indexes of unhealthy_
rng = np.random.default_rng(seed)
myRDarray = rng.integers(low = 0, high=unhealthy_augmented.shape[0], size= N , dtype=int)

#select randomly N augmented images among all with the index array
aug_data = np.array([unhealthy_augmented[i] for i in myRDarray])

# create the output array for all augmented samples (so unhealthy or [0,1])
aug_labels = np.array(['unhealthy' for i in range(N)])

#add the augmented data to the entire data set !!!

data = np.concatenate([data, aug_data], axis=0)
labels = np.concatenate([labels, aug_labels], axis=0)

print('new dataset shape : ', data.shape, 'new labelset shape', labels.shape)

number of augmented samples created  1903

number of selected images :  1198

new dataset shape :  (6202, 96, 96, 3) new labelset shape (6202,)


### Control the result

In [ ]:
#count occurences once again :

#set new setsize
setsize = labels.shape[0]
aug_ClassCount = pd.DataFrame(labels, columns=['state'])['state'].value_counts()
aug_healthyCount = aug_ClassCount.at['healthy']
aug_unhealthyCount = aug_ClassCount.at['unhealthy']

print('Counting occurrences of target classes AFTER DATA AUGMENTATION:')
print("healthy samples occurence =", aug_healthyCount)
print("unhealthy samples occurence =", aug_unhealthyCount)

print('\nTotal number of samples : ', setsize)


Counting occurrences of target classes AFTER DATA AUGMENTATION:

healthy samples occurence = 3101

unhealthy samples occurence = 3101



Total number of samples :  6202


# Prepare data

In [ ]:
#normalize and encode data

labels_map= {'healthy': 0, 'unhealthy':1}
for i in range(setsize):
  a=labels[i]
  labels[i]=labels_map[a]
y = tfk.utils.to_categorical(labels)

print(y, y.shape)

X = data.astype('uint8')

KeyError: ignored

Split the training-validation-testing sets

In [ ]:
# Split data into train_val and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=seed, test_size=900, stratify=y)

# Further split train_val into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=seed, test_size=900, stratify=np.argmax(y_train_val,axis=1))

# Print shapes of the datasets
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (4402, 96, 96, 3), y_train shape: (4402, 2)

X_val shape: (900, 96, 96, 3), y_val shape: (900, 2)

X_test shape: (900, 96, 96, 3), y_test shape: (900, 2)


In [ ]:
# define key model parameters
input_shape = X_train.shape[1:]  # Input shape for the model
output_shape = y_train.shape[1]  # Output shape for the model
batch_size = 128                # Batch size for training ------> need to experiment this
epochs = 200                     # Number of training epochs

# Print the defined parameters
print("Epochs:", epochs)
print("Batch Size:", batch_size)
print("Input Shape:", input_shape)
print("Output Shape:", output_shape)

# Build the model

In [ ]:
base_model=ResNet50(
    include_top=False,
    input_shape=(96,96,3),
    pooling='avg',
    weights='imagenet',
    classes=2,
)

# Use the supernet as feature extractor, i.e. freeze all its weigths
base_model.trainable = False

tfk.utils.plot_model(base_model, show_shapes=True)

In [ ]:
# Create an input layer with shape (96, 96, 3)

inputs = tfk.Input(shape=(96, 96, 3))

#add image augmentation
preprocessing = tf.keras.Sequential([
    tfkl.RandomTranslation(0.1,0.1, seed = seed),
    tfkl.RandomFlip(mode = "vertical", seed = seed),
    tfkl.RandomFlip(mode = 'horizontal', seed = seed)
    ], name='preprocessing')

preprocessed_inputs = preprocessing(inputs)
x = base_model(preprocessed_inputs)
x=tfkl.Dropout(0.6)(x)
x=tfkl.Dense(1024,activation='relu')(x)
x=tfkl.Dropout(0.6)(x)
outputs = tfkl.Dense(2, activation='softmax')(x)

model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics=['accuracy'])
model.summary()

In [ ]:
EarlyStopping = tfk.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max', patience=5,
    restore_best_weights=True)

callbacks = [EarlyStopping]

# Train model

In [ ]:
# Train the model
history = model.fit(
    x = preprocess_input(X_train), # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train,
    batch_size = 64,
    class_weight = {0: 0.9, 1: 1.2}, # Improve the re-balance of the dataset by telling network to pay more attention to errors on unhealthy data
    epochs = 200,
    validation_data = (preprocess_input(X_val), y_val), # We need to apply the preprocessing thought for the MobileNetV2 network
    callbacks = callbacks).history

# Save model

In [ ]:
model.save('les_manchots_tl_resnet_noFN')

# Observe results

In [ ]:
# Find the epoch with the highest validation accuracy
best_epoch = np.argmax(history['val_accuracy'])

# Plot training and validation performance metrics
plt.figure(figsize=(20, 5))

# Plot training and validation loss
plt.plot(history['loss'], label='Training', alpha=0.8, color='#ff7f0e', linewidth=3)
plt.plot(history['val_loss'], label='Validation', alpha=0.8, color='#4D61E2', linewidth=3)
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=0.3)

plt.figure(figsize=(20, 5))

# Plot training and validation accuracy, highlighting the best epoch
plt.plot(history['accuracy'], label='Training', alpha=0.8, color='#ff7f0e', linewidth=3)
plt.plot(history['val_accuracy'], label='Validation', alpha=0.8, color='#4D61E2', linewidth=3)
plt.plot(best_epoch, history['val_accuracy'][best_epoch], marker='*', alpha=0.8, markersize=10, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=0.3)

plt.show()

In [ ]:
# Extract activations from the first convolutional layer for a subset of test images
num_images = 100
first_conv = tfk.Sequential(model.layers[:2])
first_activations = first_conv(X_test[0:num_images])

# Extract activations from the second convolutional layer for the same subset of test images
second_conv = tfk.Sequential(model.layers[:4])
second_activations = second_conv(X_test[0:num_images])

In [ ]:
# Choose a random image for visualization
image = np.random.randint(0, num_images)
re_label_map={0:'healthy', 1:'unhealthy'}

# Display the input image, true label, and predicted label
print('Input image')
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.set_size_inches(18, 5)
ax1.imshow(np.clip(X_test[image], 0, 255))
print(X_test[image].shape)
ax1.set_title('True label: ' + re_label_map[np.argmax(y_test[image])])
prediction = model.predict(np.expand_dims(X_test[image], axis=0), verbose=0)
ax2.barh(list(re_label_map.values()), np.squeeze(prediction, axis=0), color=plt.get_cmap('Paired').colors)
ax2.set_title('Predicted label: ' + re_label_map[np.argmax(prediction)])
ax2.grid(alpha=0.3)
plt.show()

# Make inference

In [ ]:
# Predict labels for the entire test set
predictions = model.predict(preprocess_input(X_test), verbose=0)

# Display the shape of the predictions
print("Predictions Shape:", predictions.shape)

# Compute the confusion matrix
cm = confusion_matrix(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1))

# Compute classification metrics
accuracy = accuracy_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1))
precision = precision_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')
recall = recall_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')
f1 = f1_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')

# Display the computed metrics
print('Accuracy:', accuracy.round(4))
print('Precision:', precision.round(4))
print('Recall:', recall.round(4))
print('F1:', f1.round(4))

# Plot the confusion matrix
plt.figure(figsize=(5, 4))
sns.heatmap(cm.T, xticklabels=list(re_label_map.values()), yticklabels=list(re_label_map.values()), cmap='Blues')
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

# Fine Tuning

In [ ]:
del model
# Re-load the model after transfer learning
ft_model = tfk.models.load_model('les_manchots_tl_resnet_noFN')
ft_model.summary()

In [ ]:
# set all layers to trainable
ft_model.get_layer('resnet50').trainable = True
for i, layer in enumerate(ft_model.get_layer('resnet50').layers):
            print(i, layer.name, layer.trainable)

In [ ]:
# Freeze first N layers, e.g., until the 133rd one
N = 100
for i, layer in enumerate(ft_model.get_layer('resnet50').layers[:N]):
       layer.trainable=False
for i, layer in enumerate(ft_model.get_layer('resnet50').layers):
       print(i, layer.name, layer.trainable)
ft_model.summary()

In [ ]:
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-5), metrics='accuracy')

In [ ]:
# Train the model
ft_history = ft_model.fit(
    x = X_train,
    y = y_train,
    batch_size = 16,
    epochs = 200,
    validation_data = (X_val, y_val),
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=3, restore_best_weights=True)]
).history

In [ ]:
# Plot the transfer learning and the fine-tuned resnet training histories
plt.figure(figsize=(15,5))
plt.plot(history['loss'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(history['val_loss'], label='Re-trained', alpha=.8, color='#ff7f0e')
plt.plot(ft_history['loss'], alpha=.3, color='#408537', linestyle='--')
plt.plot(ft_history['val_loss'], label='Fine Tuning', alpha=.8, color='#408537')
plt.legend(loc='upper left')
plt.title('Binary Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(history['val_accuracy'], label='Re-trained', alpha=.8, color='#ff7f0e')

plt.plot(ft_history['accuracy'], alpha=.3, color='#408537', linestyle='--')
plt.plot(ft_history['val_accuracy'], label='Fine Tuning', alpha=.8, color='#408537')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
# Evaluate the model on the test set
test_accuracy = ft_model.evaluate(X_test,y_test,verbose=0)[-1]
print('Test set accuracy %.4f' % test_accuracy)

In [ ]:
# Predict labels for the entire test set
predictions = ft_model.predict(X_test, verbose=0)

# Display the shape of the predictions
print("Predictions Shape:", predictions.shape)

# Compute the confusion matrix
cm = confusion_matrix(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1))

# Compute classification metrics
accuracy = accuracy_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1))
precision = precision_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')
recall = recall_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')
f1 = f1_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')

# Display the computed metrics
print('Accuracy:', accuracy.round(4))
print('Precision:', precision.round(4))
print('Recall:', recall.round(4))
print('F1:', f1.round(4))

# Plot the confusion matrix
plt.figure(figsize=(6,5 ))
sns.heatmap(cm.T, xticklabels=list(re_label_map.values()), yticklabels=list(re_label_map.values()), cmap='Blues')
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

# Save final model

In [ ]:
ft_model.save('Model/les_manchots_tl_resnet_FINAL')

!zip -r file.zip /kaggle/working

!ls

from IPython.display import FileLink
FileLink(r'file.zip')